# interferentiepatroon tralie en visualisatie met phasoren
Dit programma berekent het interferentiepatroon van een tralie. In de simulatie kan de het aantal spleten N, de spleetbreedte b, de afstand tussen de spleten a en de golflengte van het licht worden ingesteld. Daarnaast wordt dit patroon gevisualiseerd door gebruik te maken van phasoren. Elke Phasor representeert het elektrisch veld van één spleet.

In [1]:
# instellen van programma
import math
import numpy as np              #importeer bibliotheek numpy
import matplotlib.pyplot as plt #importeer matplot
import cmath          #gebruik complexe getallen
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Output

In [2]:
#hier wordt de class Tralie aangemaakt, hier horen ook functies bij die berekeningen doen op dit object
class Tralie: 
    def __init__(self, N, a, b, golflengte, hoek, I_spleten, I_enkelspleet, I_totaal):
        self.N = N #aantal spleten
        self.a = a #afstand tussen spleten
        self.b = b #breedte spleten
        self.golflengte = golflengte
        self.I_s = 0
        self.I_enk = 0
        self.I_t = 0
         # hoek laten beginnen bij onderkant range, ondergrens = - bovengrens
        self.theta = - range_hoek
        
        #lijsten maken met variabelen
        self.hoek = hoek # de lijst 'hoeken' met een beginhoek gemaakt
        self.I_spleten = I_spleten # lijst  met waardes voor Intesiteit van multiple spleten gemaakt 
        self.I_enkelspleet = I_enkelspleet# en Intensiteit van alleen enkelspleet
        self.I_totaal = I_totaal # en intensiteit van totale patroon
          
     
    def bereken_Intensiteit(self):
        while self.theta < range_hoek:
            # eerst beta en alfa uitrekenen
            alfa = math.pi/self.golflengte*self.a*math.sin(self.theta)
            beta = math.pi/self.golflengte*self.b*math.sin(self.theta)
            self.I_s = 1/self.N**2 * (math.sin(self.N*alfa)/math.sin(alfa))**2
            self.I_enk = (math.sin(beta)/beta)**2
            self.I_t = self.I_s * self.I_enk
            
            # toevoegen van berekende waarden aan lijsten
            self.hoek.append(self.theta)
            self.I_spleten.append(self.I_s)
            self.I_enkelspleet.append(self.I_enk)
            self.I_totaal.append(self.I_t) 
           
            self.theta += d_theta
 
#hier kunnen diverse parameters ingesteld worden er wordt een object van de soort Tralie aangemaakt en er wordt berekend hoe het intensiteitspatroon van deze tralie eruit ziet.
def bereken_patroon_tralie(N, a, b, golflengte):
   
   #lege lijsten aanmaken die meegestuurd worden
    hoek = []
    I_spleten = []
    I_enkelspleet = []
    I_totaal = []
   
    tralie = Tralie(N, a, b, golflengte, hoek, I_spleten, I_enkelspleet, I_totaal)
    tralie.bereken_Intensiteit()
    return tralie

# Functie om de kleur te bepalen op basis van de golflengte
def golflengte_naar_kleur(golflengte_nm):
    if 380 <= golflengte_nm < 440:
        R = -(golflengte_nm - 440) / (440 - 380)
        G = 0.0
        B = 1.0
    elif 440 <= golflengte_nm < 490:
        R = 0.0
        G = (golflengte_nm - 440) / (490 - 440)
        B = 1.0
    elif 490 <= golflengte_nm < 510:
        R = 0.0
        G = 1.0
        B = -(golflengte_nm - 510) / (510 - 490)
    elif 510 <= golflengte_nm < 580:
        R = (golflengte_nm - 510) / (580 - 510)
        G = 1.0
        B = 0.0
    elif 580 <= golflengte_nm < 645:
        R = 1.0
        G = -(golflengte_nm - 645) / (645 - 580)
        B = 0.0
    elif 645 <= golflengte_nm <= 700:
        R = 1.0
        G = 0.0
        B = 0.0
    else:
        R = G = B = 0.0
    
    return (R, G, B)

In [3]:
# in dit deel worden functies gedefinieerd die de gegeven plotten.

# in deze functies worden de phasors berekend en geplot. Deze functie wordt aangeroepen in de functie update_plot
def plot_phasors(N, a, golflengte, hoek_phasors):
    phasor_som = 0
    phasors = []
    #voor alle phasoren wordt de hoek ten opzichte van de vorige phasor berekend, deze worden opgeslagen als complex getal met modulus 1
    for i in range(N):
        hoek = i * math.sin(hoek_phasors)*a/golflengte*2*math.pi
        phasor = cmath.rect(1, hoek)
        phasor_som += phasor
        phasors.append(phasor)
    
      # hier wordt een plot met naast elkaar de twee phasoren plots gemaakt
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    u = [phasor.real for phasor in phasors]
    v = [phasor.imag for phasor in phasors]

  #hier worden de losse phasoren van elke spleet getekend
    for i in range(len(phasors)):
        ax1.quiver(0, 0, u[i], v[i], angles='xy', scale_units='xy', scale=1, color='blue')
        
      # Teken assenstelsel
    max_val = 0.5
    ax1.set_xlim(-max_val-1, max_val+1)
    ax1.set_ylim(-max_val-1, max_val+1)
    ax1.axhline(0, color='black',linewidth=1)
    ax1.axvline(0, color='black',linewidth=1)
    ax1.set_title('Losse Phasors van de N spleten' )
    ax1.grid()
    ax1.set_aspect(1, adjustable='box')
        
    
     #hier worden de phasoren van elke spleet kop-staart getekend
    start_x, start_y, max_x, max_y = 0, 0, 0, 0
    for i in range(len(phasors)):
        ax2.quiver(start_x, start_y, u[i], v[i], angles='xy', scale_units='xy', scale=1, color='blue')
        start_x += u[i]
        start_y += v[i]
        max_x = max(max_x, start_x)
        max_y = max(max_y, start_y)
        
    #tekenen van de resulterende phasor
    ax2.quiver(0, 0, phasor_som.real, phasor_som.imag, angles='xy', scale_units='xy', scale=1, color='red')

      # Teken assenstelsel
    max_val = max(max_x, max_y)
    ax2.set_xlim(-max_val-1, max_val+1)
    ax2.set_ylim(-max_val-1, max_val+1)
    ax2.axhline(0, color='black',linewidth=1)
    ax2.axvline(0, color='black',linewidth=1)
    ax2.set_title('Optellen van Phasors van de N spleten' )
    ax2.grid()
    ax2.set_aspect(1, adjustable='box')
    plt.show()

def update_plot(change):
    N = slider_N.value
    golflengte = slider_golflengte.value * 1E-9
    a = slider_a.value * 1E-6
    b = a / slider_ab.value
    
    tralie_start = bereken_patroon_tralie(start_N, start_a * 1E-6, start_b * 1E-6, start_golflengte * 1E-9)
    tralie_current_values = bereken_patroon_tralie(N, a, b, golflengte)

    lijn_kleur_1 = golflengte_naar_kleur(start_golflengte)
    lijn_kleur_2 = golflengte_naar_kleur(slider_golflengte.value)

    with output:
        clear_output(wait=True)
        fig, (ax1) = plt.subplots(figsize=(10, 10))
            # hier worden de waardes voor de twee tralies geplot
        ax1.plot(tralie_start.hoek, tralie_start.I_totaal, label=f'aantal spleten = {start_N}, a = {start_a:.2f} µm, b = {start_b:.2f} µm, λ = {start_golflengte:.2f} nm', color=lijn_kleur_1, linestyle='dotted')
        ax1.plot(tralie_current_values.hoek, tralie_current_values.I_totaal, label=f'aantal spleten = {N}, a = {slider_a.value:.2f} µm, b = {b * 1E6:.2f} µm, λ = {slider_golflengte.value:.2f} nm', color=lijn_kleur_2)
          #de vertical lijn die de gekozen waarde voor de hoek aangeeft wordt geplot
        ax1.axvline(x=slider_hoek.value, color='black', linestyle='--')
        ax1.set_title('Diffractie patroon tralie')
        ax1.set_xlabel('hoek (radialen)')
        ax1.set_ylabel('relatieve intensiteit')
        ax1.set_ylim(bottom=0)
        ax1.legend(loc='lower left', bbox_to_anchor=(1, 0.5))
        ax1.grid()
        ax1.set_aspect(0.5, adjustable='box') #hier wordt de vaste verhouding van tussen de x- en y-as van de plot gegeven
        ax1.minorticks_on()
        ax1.grid(which='both', linestyle='--', linewidth=0.5)

        plt.show()
            # Voeg de slider_hoek toe onder de grafiek met I_totaal
        display(widgets.HBox([widgets.Label('Hoek (radialen):', layout=widgets.Layout(width='270px')), slider_hoek]))
        
        #hier worden de phasors getekend en geplot
        plot_phasors(N, a, golflengte, slider_hoek.value)
  

In [4]:
#hier begint het programma, er worden eerst een aantal parameters ingesteld
# instellen hoek en stappen in hoek
d_theta = 0.0001
range_hoek = 0.3 #begin en eindhoek in radialen

#standaard waarden voor N, a, b en golflengte meegeven, deze wordt als standaard weergegeven in de plot naast de plot
#naast de plot met waardes die je in de sliders kunt instellen
start_N= 2
start_a = 6
start_golflengte = 590
start_ab = 3
start_b = start_a/start_ab

# het aantal spleten N later varieren met een slider
slider_N = widgets.IntSlider( value=start_N,min=1,max=20, step=1, continuous_update=False)
#spleetafstand varieren met slider
slider_a = widgets.FloatSlider(value=start_a, min=1, max=10, step=0.5, readout_format='.2f', continuous_update=False)
# golflengte varieren met slider
slider_golflengte = widgets.FloatSlider(value=start_golflengte, min=400, max=700, step=10, readout_format='.0f', continuous_update=False)
# verhouding spleetafstand en spleetbreedte varieren met slider
slider_ab = widgets.FloatSlider(value=start_ab, min=2, max=10, step=0.5, readout_format='.1f', continuous_update=False)
# slider om de hoek te kiezen
slider_hoek = widgets.FloatSlider(value=0, min=-range_hoek, max=range_hoek, step=0.001, readout_format='.3f',continuous_update=False)

# Maak een Output widget
output = widgets.Output()
# gelijk een eerste grafiek tekenen door de functie update_plot aan te roepen 
update_plot({'new': slider_N.value})

#sliders tekenen
display(widgets.VBox([widgets.HBox([widgets.Label('aantal spleten N:', layout=widgets.Layout(width='270px')), slider_N]),
                      widgets.HBox([widgets.Label('Spleetafstand a (µm):', layout=widgets.Layout(width='270px')), slider_a]),
                      widgets.HBox([widgets.Label('Verhouding spleetafstand a/spleetbreedte b:', layout=widgets.Layout(width='270px')), slider_ab]),
                      widgets.HBox([widgets.Label('Golflengte (nm):', layout=widgets.Layout(width='270px')), slider_golflengte]),
                      output,
                      ]))

#testen of er een verandering is van de sliders, als dit zo is plots updaten                 
slider_N.observe(update_plot, names='value')
slider_a.observe(update_plot, names='value')
slider_golflengte.observe(update_plot, names='value')
slider_ab.observe(update_plot, names='value')
slider_hoek.observe(update_plot, names='value')
